# NIH awards data
https://exporter.nih.gov/ExPORTER_Catalog.aspx

In [766]:
import requests, zipfile, io
from bs4 import BeautifulSoup
import os

import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [767]:
url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2016.zip'
r = requests.get(url).content
z = zipfile.ZipFile(io.BytesIO(r))
z.extractall()

csv = 'RePORTER_PRJ_C_FY2016.csv'
grants_2016_raw = pd.read_csv(csv, encoding = 'latin1')
os.remove(csv)

pd.set_option('display.max_columns', 50)
grants_2016_raw.head(10)

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,ED_INST_TYPE,FOA_NUMBER,FULL_PROJECT_NUM,FUNDING_ICs,FUNDING_MECHANISM,FY,IC_NAME,NIH_SPENDING_CATS,ORG_CITY,ORG_COUNTRY,ORG_DEPT,ORG_DISTRICT,ORG_DUNS,ORG_FIPS,ORG_NAME,ORG_STATE,ORG_ZIPCODE,PHR,PI_IDS,PI_NAMEs,PROGRAM_OFFICER_NAME,PROJECT_START,PROJECT_END,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN
1,9128072,R01,NS,4.0,N,8/15/2016,8/1/2016,7/31/2017,853,R01NS085165,SCHOOLS OF MEDICINE,PA-11-260,4R01NS085165-04,NINDS:335781\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF NEUROLOGICAL DISORDERS A...,Biotechnology; Complementary and Alternative M...,BALTIMORE,UNITED STATES,ANESTHESIOLOGY,7.0,188435911,US,UNIVERSITY OF MARYLAND BALTIMORE,MD,212011508,PUBLIC HEALTH RELEVANCE: Activation of microgl...,7017365;,"POLSTER, BRIAN M;","MORRIS, JILL A",9/30/2013,7/31/2018,Acute; analog; Antioxidants; attenuation; Bind...,Novel Mechanisms of Microglial Neurotoxicity a...,85165.0,NOMD,Neural Oxidative Metabolism and Death Study Se...,NaN,NaN,4.0,218750.0,117031.0,335781.0,NaN
2,9056435,D43,TW,4.0,N,3/19/2016,3/1/2016,2/28/2017,989,D43TW009310,SCHOOLS OF MEDICINE,PAR-10-257,4D43TW009310-05,FIC:146822\NIDA:75000\,OTHER RESEARCH-RELATED,2016,FOGARTY INTERNATIONAL CENTER,Aging; Behavioral and Social Science; Brain Di...,ANN ARBOR,UNITED STATES,PSYCHIATRY,12.0,73133571,US,UNIVERSITY OF MICHIGAN,MI,481091276,Although recent science has documented the dev...,7354078; 1862210 (contact);,"WALTON, MAUREEN A; ZUCKER, ROBERT ALPERT (cont...","MICHELS, KATHLEEN M",7/9/2012,2/28/2017,21 year old; Academy; addiction; Address; Alco...,Capacity Building for Lifespan Focused Substan...,9310.0,ZRG1,Special Emphasis Panel,NaN,NaN,5.0,209478.0,12344.0,221822.0,NaN
3,9213716,R01,MH,1.0,N,9/8/2016,9/9/2016,5/31/2017,242,R01MH111502,SCHOOLS OF ARTS AND SCIENCES,RFA-MH-16-775,1R01MH111502-01,NIMH:1593922\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Basic Behavioral and Social Science; Behaviora...,CAMBRIDGE,UNITED STATES,MICROBIOLOGY/IMMUN/VIROLOGY,5.0,82359691,US,HARVARD UNIVERSITY,MA,21385369,Project Narrative To shed light into the basic...,2275890 (contact); 8742217; 6139020;,"DULAC, CATHERINE G (contact); REGEV, AVIV ; ZH...","FREUND, MICHELLE",9/9/2016,5/31/2019,abstracting; Accounting; Address; Aggressive b...,In situ transcriptional analysis of brain circ...,111502.0,ZMH1,Special Emphasis Panel,NaN,NaN,1.0,1077309.0,516613.0,1593922.0,NaN
4,8986215,R01,MH,5.0,N,11/5/2015,12/1/2015,11/30/2016,242,R01MH105329,SCHOOLS OF MEDICINE,PA-14-094,5R01MH105329-02,NIMH:294755\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Acquired Cognitive Impairment; Brain Disorders...,GRAND FORKS,UNITED STATES,OTHER BASIC SCIENCES,0.0,102280781,US,UNIVERSITY OF NORTH DAKOTA,ND,582026059,PUBLIC HEALTH RELEVANCE: The proposed research...,9868481; 2063694 (contact);,"CHEN, XUESONG ; GEIGER, JONATHAN DAVID (contact);","JOSEPH, JEYMOHAN",12/15/2014,11/30/2019,Affect; Anti-Retroviral Agents; antiretroviral...,Role of novel endolysosome-dependent calcium r...,105329.0,ZRG1,Special Emphasis Panel,NaN,NaN,2.0,212500.0,82255.

In [768]:
grants_2016_raw.shape

(71872, 45)

# Processing data

## Basic text processing of column names

In [769]:
grants_2016 = grants_2016_raw.copy()

grants_2016.columns = grants_2016.columns.str.lower()
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


In [770]:
#converts df to lowercase string
grants_2016 = grants_2016.apply(lambda x: x.astype(str).str.lower())
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,nan,nan,4.0,180500.0,13960.0,194460.0,nan


In [771]:
#converts 'nan' to NaN
grants_2016.replace(['nan'], np.nan, inplace=True)
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


## Selecting columns for analysis
There are a total of 45 columns, which may contain redundant or unnecessary information.

In [772]:
pd.set_option('display.max_colwidth', 5000)
col_info = pd.read_csv('grant_col_info_all.csv')
col_info[:9]

,column_name,descriptions
0,application_id,A unique identifier of the project record in the ExPORTER database.
1,activity,"A 3-character code identifying the grant, contract, or intramural activity through which a project is supported. Within each funding mechanism , NIH uses 3-character activity codes (e.g., F32, K08, P01, R01, T32, etc.) to differentiate the wide variety of research-related programs NIH supports. A comprehensive list of activity codes for grants and cooperative agreements may be found on the Types of Grant Programs Web page. RePORTER also includes R&D contracts (activity codes beginning with the letter N) and intramural projects (beginning with the letter Z)."
2,administering_ic,"Administering Institute or Center - A two-character code to designate the agency,NIH Institute, or Center administering the grant. See Institute/Center code definitions"
3,application_type,"A one-digit code to identify the type of application funded: 1 = New application 2 = Competing continuation (also, competing renewal) 3 = Application for additional (supplemental) support. There are two kinds of type 3competing revisions (which are peer-reviewed and administrative supplements) 4 = Competing extension for an R37 award or first non-competing year of a Fast Track SBIR/STTR award 5 = Non-competing continuation 7 = Change of grantee institution 9 = Change of NIH awarding Institute or Division (on a competing continuation)"
4,arra_funded,“Y” indicates a project supported by funds appropriated through the American Recovery and Reinvestment Act of 2009.
5,award_notice_date,Award notice date or Notice of Grant Award (NGA) is a legally binding document stating the government has obligated funds and which defines the period of support and the terms and conditions of award.\r\n
6,budget_start,The date when a project’s funding for a particular fiscal year begins.
7,budget_end,The date when a project’s funding for a particular fiscal year ends.
8,cfda_code,"Federal programs are assigned a number in the Catalog of Federal Domestic Assistance (CFDA), which is referred to as the ""CFDA code."" The CFDA database helps the Federal government track all programs it has domestically funded. \r\n"


To remove (initial): 1, 5-8, redundant information

In [773]:
to_drop = ['application_id', 'award_notice_date', 'budget_start', 'budget_end', 'cfda_code']
grants_2016 = grants_2016.drop(to_drop, axis = 1)

In [774]:
col_info[9:18]

,column_name,descriptions
9,core_project_num,"An identifier for each research project, used to associate the project with publication and patent records. This identifier is not specific to any particular year of the project. It consists of the project activity code, administering IC, and serial number (a concatenation of Activity, Administering_IC, and Serial_Number). \r\n"
10,ed_inst_type,Generic name for the grouping of components across an institution who has applied for or receives NIH funding. The official name as used by NIH is Major Component Combining Name. \r\n
11,foa_number,"The number of the funding opportunity announcement, if any, under which the project application was solicited. Funding opportunity announcements may be categorized as program announcements, requests for applications, notices of funding availability, solicitations, or other names depending on the agency and type of program. Funding opportunity announcements can be found at Grants.gov/FIND and in the NIH Guide for Grants and Contracts"
12,full_project_num,"Commonly referred to as a grant number, intramural project, or contract number. For grants, this unique identification number is composed of the type code, activity code, Institute/Center code, serial number, support year, and (optional) a suffix code to designate amended applications and supplements."
13,funding_ic(s),"The NIH Institute or Center(s) providing funding for a project are designated by their acronyms (see Institute/Center acronyms ). Each funding IC is followed by a colon (:) and the amount of funding provided for the fiscal year by that IC. Multiple ICs are separated by semicolons (;). Project funding information is available only for NIH, CDC, and FDA projects ."
14,funding_mechanism,"The major mechanism categories used in NIH Budget mechanism tables for the President’s budget. Extramural research awards are divided into three main funding mechanisms: grants, cooperative agreements and contracts. A funding mechanism is the type of funded application or transaction used at the NIH. Within each funding mechanism NIH includes programs. Programs can be further refined by specific activity codes."
15,fy,The fiscal year appropriation from which project funds were obligated.
16,ic_name,"Full name of the administering agency, Institute, or Center."
17,nih_spending_cats,"Congressionally-mandated reporting categories into which NIH projects are categorized. Available for fiscal years 2008 and later. Each project’s spending category designations for each fiscal year are made available the following year as part of the next President’s Budget request. See the Research, Condition, and Disease Categorization System for more information on the categorization process."


To remove: 9 (may add in later if correlating with publications), 10, 11, 16; also 2 (redundant with funding_ics)

In [775]:
to_drop2 = ['administering_ic', 'core_project_num', 'ed_inst_type', 'foa_number', 'ic_name']
grants_2016 = grants_2016.drop(to_drop2, axis = 1)

In [776]:
col_info[18:27]

,column_name,descriptions
18,org_city,"The city in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site. For all NIH intramural projects, Bethesda, MD is used."
19,org_country,The country in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
20,org_dept,"The departmental affiliation of the contact principal investigator for a project, using a standardized categorization of departments. Names are available only for medical school departments."
21,org_district,The congressional district in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
22,org_duns,"This field may contain multiple DUNS Numbers separated by a semi-colon. The Data Universal Numbering System is a unique nine-digit number assigned by Dun and Bradstreet Information Services, recognized as the universal standard for identifying and keeping track of business worldwide. \r\n"
23,org_fips,The country code of the grantee organization or contractor as defined in the Federal Information Processing Standard.
24,org_name,"The name of the educational institution, research organization, business, or government agency receiving funding for the grant, contract, cooperative agreement, or intramural project."
25,org_state,The state in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
26,org_zipcode,The zip code in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.


All redundant: 19-23, 26

In [777]:
to_drop3 = ['org_country', 'org_dept', 'org_district', 'org_duns', 'org_fips', 'org_zipcode']
grants_2016 = grants_2016.drop(to_drop3, axis = 1)

In [778]:
col_info[27:36]

,column_name,descriptions
27,phr,"Submitted as part of a grant application, this statement articulates a project's potential to improve public health."
28,pi_id(s),A unique identifier for each of the project Principal Investigators. Each PI in the RePORTER database has a unique identifier that is constant from project to project and year to year.
29,pi_name(s),The name(s) of the Principal Investigator(s) designated by the organization to direct the research project.
30,program_officer_name,An Institute staff member who coordinates the substantive aspects of a contract from planning the request for proposal to oversight.
31,project_start,"The start date of a project. For subprojects of a multi-project grant, this is the start date of the parent award."
32,project_end,"The current end date of the project, including any future years for which commitments have been made. For subprojects of a multi-project grant, this is the end date of the parent award. Upon competitive renewal of a grant, the project end date is extended by the length of the renewal award."
33,project_terms,"Prior to fiscal year 2008, these were thesaurus terms assigned by NIH CRISP indexers. For projects funded in fiscal year 2008 and later, these are concepts that are mined from the project's title, abstract, and specific aims using an automated text mining tool."
34,project_title,"Title of the funded grant, contract, or intramural (sub)project."
35,serial_number,A six-digit number assigned in serial number order within each administering organization.


27 (phr): maybe later for text analysis, too complex for initial analysis; 28, 34, 35

In [779]:
to_drop4 = ['phr', 'pi_ids', 'project_title', 'serial_number']
grants_2016 = grants_2016.drop(to_drop4, axis = 1)

In [780]:
col_info[36:]

,column_name,descriptions
36,study_section,A designator of the legislatively-mandated panel of subject matter experts that reviewed the research grant application for scientific and technical merit.
37,study_section_name,The full name of a regular standing Study Section that reviewed the research grant application for scientific and technical merit. Applications reviewed by panels other than regular standing study sections are designated by “Special Emphasis Panel.”
38,subproject_id,A unique numeric designation assigned to subprojects of a “parent” multi-project research grant.
39,suffix,"A suffix to the grant application number that includes the letter ""A"" and a serial number to identify an amended version of an original application and/or the letter ""S"" and serial number indicating a supplement to the project. ."
40,support_year,"The year of support for a project, as shown in the full project number. For example, a project with number 5R01GM0123456-04 is in its fourth year of support."
41,direct_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
42,indirect_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
43,total_cost,"Total project funding from all NIH Institute and Centers for a given fiscal year. Costs are available only for: NIH, CDC, and FDA grant awards (only the parent record of multi-project grants). -NIH intramural projects (activity codes beginning with “Z”) in FY 2007 and later fiscal years. -NIH contracts (activity codes beginning with “N”) . For multi-project grants, Total_Cost includes funding for all of the constituent subprojects. This field will be blank on subproject records; the total cost of each subproject is found in Total_Cost_Sub_Project ."
44,total_cost_sub_project,Applies to subproject records only. Total funding for a subproject from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards.


37 (redundant with 36, can always look up)
Maybe 38 and 39, depending on how many instances there are

In [781]:
#check number of subprojects
grants_2016.shape
subproject_cols = ['subproject_id', 'suffix', 'total_cost_sub_project']
grants_2016[subproject_cols].isnull().sum()

(71872, 25)

subproject_id             60759
suffix                    61151
total_cost_sub_project    60759
dtype: int64

About 85% of the grants do not have any entries for these three columns; drop for now, along with 37.

In [782]:
grants_2016 = grants_2016.drop(['study_section_name'] + subproject_cols, axis = 1)

In [783]:
pd.set_option('display.max_colwidth', 50)
grants_2016.head()

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,pi_names,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost
0,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,new york,icahn school of medicine at mount sinai,ny,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0
1,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,baltimore,university of maryland baltimore,md,"polster, brian m;","morris, jill a",9/30/2013,7/31/2018,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0
2,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,ann arbor,university of michigan,mi,"walton, maureen a; zucker, robert alpert (cont...","michels, kathleen m",7/9/2012,2/28/2017,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0
3,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cambridge,harvard university,ma,"dulac, catherine g (contact); regev, aviv ; zh...","freund, michelle",9/9/2016,5/31/2019,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0
4,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,grand forks,university of north dakota,nd,"chen, xuesong ; geiger, jonathan david (contact);","joseph, jeymohan",12/15/2014,11/30/2019,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0


Convert strings to numeric and dates

In [784]:
def convert_column(df, list_of_names, function):
    '''
    Converts df column values from x to 'function'.
    Function = numeric: pd.to_numeric()
    Function = date: pd.to_datetime()
    Takes a df and a list with the column names.
    Returns df.
    '''
    for name in list_of_names:
        if function == 'numeric':
            df[name] = pd.to_numeric(df[name])
        elif function == 'date':
            df[name] = pd.to_datetime(df[name], errors = 'coerce')
    return df

numeric_cols = ['direct_cost_amt', 'indirect_cost_amt', 'total_cost']
grants_2016 = convert_column(grants_2016, numeric_cols, function = 'numeric')

date_cols = ['project_start', 'project_end']
grants_2016 = convert_column(grants_2016, date_cols, function = 'date')


grants_2016.head(1)

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,pi_names,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost
0,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,new york,icahn school of medicine at mount sinai,ny,"scott, stuart alexander;","long, rochelle m.",2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0


Names are in the format last_name, first_name (optional middle name/initial).
Remove commas, spaces and periods

In [785]:
def replace_names(df, col_name, *args):
    for arg in args:
        df[col_name] = df[col_name].str.replace(args, '_')
    return df

def strip_names(df, col_name, *args):
    for arg in args:
        df[col_name] = df[col_name].str.strip(arg)
    return df

name_cols = ['pi_names', 'program_officer_name']
for name in name_cols:
    grants_2016 = replace_names(grants_2016, name, ', ', ' ')
    grants_2016 = strip_names(grants_2016, name, ';', '.')

In [786]:
grants_2016.head(10)

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,pi_names,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost
0,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,new york,icahn school of medicine at mount sinai,ny,scott_stuart_alexander,long_rochelle_m,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0
1,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,baltimore,university of maryland baltimore,md,polster_brian_m,morris_jill_a,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0
2,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,ann arbor,university of michigan,mi,walton_maureen_a;_zucker_robert_alpert_(contact),michels_kathleen_m,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0
3,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cambridge,harvard university,ma,dulac_catherine_g_(contact);_regev_aviv_;_zhua...,freund_michelle_,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0
4,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,grand forks,university of north dakota,nd,chen_xuesong_;_geiger_jonathan_david_(contact),joseph_jeymohan_,2014-12-15,2019-11-30,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0
5,p41,4.0,n,4p41eb015871-30,nibib:992896\,research centers,2016,bioengineering; biotechnology; cancer; regener...,cambridge,massachusetts institute of technology,ma,so_peter_t.,shabestari_behrouz_,1997-06-01,2017-05-31,area; automobile driving; biological research;...,zrg1,30.0,630065.0,362831.0,992896.0
6,u01,5.0,n,5u01ns094288-02,ninds:627714\od:500000\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cold spring harbor,cold spring harbor laboratory,ny,eden_uri_tzvi;_frank_loren_m;_ganguli_surya_;_...,gnadt_james_w,2015-09-30,2018-08-31,adaptive behaviors; address; area; auditory; b...,zns1,2.0,945822.0,181892.0,1127714.0
7,dp5,5.0,n,5dp5od021391-02,nci:399999\nidcr:1\,non-sbir/sttr rpgs,2016,bioengineering; cancer; clinical research; den...,philadelphia,university of pennsylvania,pa,gade_terence_p,basavappa_ravi_,2015-09-25,2020-08-31,"2,4-dinitrophenol; acetates; acids; alpha keto...",zrg1,2.0,250000.0,150000.0,400000.0
8,r01,5.0,n,5r01dk105393-02,niddk:202611\,non-sbir/sttr rpgs,2016,diabetes; nutrition; obesity;,ithaca,cornell university,ny,qi_ling_,haft_carol_r,2015-04-11,2016-07-31,accounting; adaptor signaling protein; adipocy...,cado,2.0,130717.0,71894.0,202611.0
9,zia,1.0,n,1ziatr000230-02,ncats:226982\,intramural research,2016,biotechnology; cancer;,NaN,translational science,NaN,boxer_matthew_,NaN,NaT,NaT,analog; antineoplastic agents; assay developme...,NaN,2.0,NaN,NaN,226982.0


**Will have to do a final cleaning of names after splitting by PI names to get rid of extra _ and semicolons**

Also, for splitting rows by PI, it will be easier to have the pi_names column as the last column

In [787]:
temp = grants_2016['pi_names']
grants_2016.drop(labels = ['pi_names'], axis = 1, inplace = True)
grants_2016['pi_names'] = temp
grants_2016.head(1)

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_names
0,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,new york,icahn school of medicine at mount sinai,ny,long_rochelle_m,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,scott_stuart_alexander


In [804]:
grants_2016.ix[grants_2016['pi_names'].str.contains('stevens_beth')]
grants_2016.ix[grants_2016['full_project_num'].str.contains('5r01ns084298-03')]
grants_2016_raw.ix[grants_2016_raw['FULL_PROJECT_NUM'].str.contains('5R01NS084298-03')]

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_names
959,r01,5.0,n,5r01ns092578-02,ninds:586541\,non-sbir/sttr rpgs,2016,brain disorders; huntington's disease; neurode...,boston,children's hospital corporation,ma,miller_daniel_l,2015-03-15,2020-02-29,address; adult; affect; age; apoptotic; autoim...,zrg1,2.0,331379.0,255162.0,586541.0,stevens_beth
19168,u54,1.0,n,1u54hd090255-01,nichd:126066\,research centers,2016,autism; basic behavioral and social science; b...,boston,children's hospital corporation,ma,NaN,2016-09-23,2021-05-31,affect; age; animal behavior; area; autism spe...,zhd1,1.0,71224.0,54842.0,NaN,stevens_beth
41069,r01,5.0,n,5r01ns084298-03,ninds:353543\,non-sbir/sttr rpgs,2016,brain disorders; huntington's disease; neurode...,los angeles,university of california los angeles,ca,miller_daniel_l,2014-07-01,2019-06-30,address; adult; age related; animal model; ant...,cdin,3.0,274740.0,78803.0,353543.0,stevens_beth
61972,r01,4.0,n,4r01ns071008-06,ninds:380625\,non-sbir/sttr rpgs,2016,neurosciences;,boston,children's hospital corporation,ma,morris_jill_a,2011-09-01,2017-07-31,affect; array tomography; astrocytes; autistic...,cmbg,6.0,218750.0,161875.0,380625.0,stevens_beth


,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_names
41069,r01,5.0,n,5r01ns084298-03,ninds:353543\,non-sbir/sttr rpgs,2016,brain disorders; huntington's disease; neurode...,los angeles,university of california los angeles,ca,miller_daniel_l,2014-07-01,2019-06-30,address; adult; age related; animal model; ant...,cdin,3.0,274740.0,78803.0,353543.0,stevens_beth
41069,r01,5.0,n,5r01ns084298-03,ninds:353543\,non-sbir/sttr rpgs,2016,brain disorders; huntington's disease; neurode...,los angeles,university of california los angeles,ca,miller_daniel_l,2014-07-01,2019-06-30,address; adult; age related; animal model; ant...,cdin,3.0,274740.0,78803.0,353543.0,yang_xiangdong_william


,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,ED_INST_TYPE,FOA_NUMBER,FULL_PROJECT_NUM,FUNDING_ICs,FUNDING_MECHANISM,FY,IC_NAME,NIH_SPENDING_CATS,ORG_CITY,ORG_COUNTRY,ORG_DEPT,ORG_DISTRICT,ORG_DUNS,ORG_FIPS,ORG_NAME,ORG_STATE,ORG_ZIPCODE,PHR,PI_IDS,PI_NAMEs,PROGRAM_OFFICER_NAME,PROJECT_START,PROJECT_END,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
41069,9099977,R01,NS,5.0,N,7/18/2016,7/1/2016,6/30/2017,853,R01NS084298,SCHOOLS OF MEDICINE,PA-13-302,5R01NS084298-03,NINDS:353543\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF NEUROLOGICAL DISORDERS A...,Brain Disorders; Huntington's Disease; Neurode...,LOS ANGELES,UNITED STATES,NONE,33.0,92530369,US,UNIVERSITY OF CALIFORNIA LOS ANGELES,CA,900952000,PUBLIC HEALTH RELEVANCE: Huntington's disease ...,10068326; 8330449 (contact);,"STEVENS, BETH ANN; YANG, XIANGDONG WILLIAM (co...","MILLER, DANIEL L",7/1/2014,6/30/2019,Address; Adult; age related; Animal Model; Ant...,Complement-Microglia Interaction in Synaptic L...,84298.0,CDIN,Cell Death in Neurodegeneration Study Section,NaN,NaN,3.0,274740.0,78803.0,353543.0,NaN


## Split rows so each row is represented by a single PI

Some grants have two or more PIs. Split these rows so that each PI is represented individually. Will also have to split:
* Cost sections, if present
* Institution, if different between the individuals, is listed as that of the main contact

In [788]:
def split_rows_pis(df, col_name = 'pi_names'):
    '''
    Different PIs are delimited by a ';'.
    Split on ';'
    '''
    s = df[col_name].str.split(';').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = col_name
    del df[col_name]
    return df.join(s)

grants_2016 = split_rows_pis(grants_2016)

In [793]:
grants_2016 = strip_names(grants_2016, 'pi_names', '(contact)', '.', '_')
grants_2016 = strip_names(grants_2016, 'program_officer_name', '_')

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_names
0,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,new york,icahn school of medicine at mount sinai,ny,long_rochelle_m,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,scott_stuart_alexander
1,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,baltimore,university of maryland baltimore,md,morris_jill_a,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,polster_brian_m
2,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,ann arbor,university of michigan,mi,michels_kathleen_m,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,walton_mauree
2,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,ann arbor,university of michigan,mi,michels_kathleen_m,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,zucker_robert_alper
3,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cambridge,harvard university,ma,freund_michelle,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,dulac_catherine_g
3,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cambridge,harvard university,ma,freund_michelle,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,regev_aviv
3,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,cambridge,harvard university,ma,freund_michelle,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,zhuang_xiaowei
4,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,grand forks,university of north dakota,nd,joseph_jeymohan,2014-12-15,2019-11-30,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0,hen_xuesong
4,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,grand forks,university of north dakota,nd,joseph_jeymohan,2014-12-15,2019-11-30,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0,geiger_jonathan_david
5,p41,4.0,n,4p41eb015871-30,nibib:992896\,research centers,2016,bioengineering; biotechnology; cancer; regener...,cambridge,massachusetts institute of technology,ma,shabestari_behrouz,1997-06-01,2017-05-31,area; automobile driving; biological research;...,zrg1,30.0,630065.0,362831.0,992896.0,so_peter


In [795]:
grants_2016.to_csv('intermediate.csv', index = False, compression = 'gzip')

# Initial data exploration

In [654]:
#count all NaNs
grants_2016.isnull().sum()

activity                   17
application_type         1627
arra_funded                17
full_project_num            0
funding_ics               155
funding_mechanism          33
fy                          0
nih_spending_cats        9490
org_city                 3345
org_name                  106
org_state                4132
pi_names                    0
program_officer_name    18412
project_start           13212
project_end             12982
project_terms            2803
study_section            5499
support_year             1627
direct_cost_amt         10674
indirect_cost_amt       15920
total_cost              13146
dtype: int64

In [672]:
#len(grants_2016.ix[grants_2016['program_officer_name'] == 'fuldner, rebecca a.'])
grants_2016['program_officer_name'].value_counts()

rankin, tracy l.               261
agarwal, neeraj                224
preusch, peter                 220
bender, michael t.             220
lees, robert g.                218
gondre-lewis, timothy a.       215
densmore, christine l          210
chin, jean                     207
harabin, andrea l              205
rivera-rentas, alberto l       203
gindhart, joseph g             203
saslowsky, david e             196
castle, arthur                 194
morris, jill a                 189
meadows, tawanna               188
ainsztein, alexandra m         187
barski, oleg                   185
stewart, randall r             184
lathrop, david a.              183
spain, lisa m                  182
wagster, molly v.              181
colombini-hatch, sandra        181
mckie, george ann              180
wehrle, janna p.               179
forry, suzanne l               177
yang, austin jyan-yu           175
fabian, miles                  174
gerratana, barbara             172
riddle, robert d.   

In [729]:
#search pi by name, sum total grant costs
pi = grants_2016.ix[grants_2016['pi_names'].str.contains('schwarz_thomas')]
pi
pi['total_cost'].sum()

,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_city,org_name,org_state,program_officer_name,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_names
16331,r01,5.0,n,5r01gm069808-11,nigms:336300\,non-sbir/sttr rpgs,2016,NaN,boston,children's hospital corporation,ma,gindhart_joseph_g,2004-09-20,2018-08-31,adaptor signaling protein; axon; base; behavio...,syn,11.0,190000.0,146300.0,336300.0,schwarz_thomas_l.
22075,r13,1.0,n,1r13ns098667-01,ninds:20000\,other research-related,2016,brain disorders; neurosciences;,west kingston,gordon research conferences,ri,stewart_randall_r,2016-06-15,2017-05-31,abstracting; asia; attention; autistic disorde...,nsd,1.0,20000.0,NaN,20000.0,schwarz_thomas_l.
36205,t32,5.0,n,5t32ns007473-17,ninds:232926\,"training, institutional",2016,neurosciences;,boston,children's hospital corporation,ma,korn_stephen_j,1999-07-01,2020-06-30,development; neurology;,zns1,17.0,215672.0,17254.0,232926.0,schwarz_thomas_l.
36838,r01,4.0,n,4r01ns041062-17,ninds:562906\,non-sbir/sttr rpgs,2016,genetics; neurosciences;,boston,children's hospital corporation,ma,miller_daniel_l,2000-04-01,2018-04-30,address; anatomy; area; base; binding; biochem...,syn,17.0,318026.0,244880.0,562906.0,schwarz_thomas_l.


1152132.0

In [ ]:
#geospatial data